In [ ]:
#importing packages
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
import matplotlib.pyplot as plt
import os
import torchvision.transforms as transforms

In [ ]:
#configure device
device='cuda' if torch.cuda.is_available() else 'cpu'
print(device)

In [ ]:
#define hyper parameters
num_epochs=3
batch_size=100
learning_rate=0.001
input_dim=90
hidden_dim=56
output_dim=1


In [ ]:
os.mkdir("mydata")

In [ ]:
#cleaning data and selecting features
data_filepath="../input/nifty50-stock-market-data/"
for i in os.listdir(data_filepath):
    df=pd.read_csv(data_filepath+i)
    df=df.dropna()
    if len(df)==2456:
        #o=df["Open"]
        #c=df["Close"]
        #l=df["Low"]
        #h=df["High"]
        #d=df["%Deliverble"]
        #df["oc"]=o-c
        #df["ho"]=h-o
        #df["ol"]=o-l
        #df["gap"]=o-df["Prev Close"]
        df2=df[["Close","Open","Prev Close"]]
        df2.to_csv("mydata/"+i)
        print("file added")


In [ ]:
print(df2.head())
print(df2.iloc[2455])

In [ ]:
x=os.listdir("mydata/")

print(len(x))

In [ ]:
print(df2.iloc[1].shape)

In [ ]:
#creating custom dataset
class Mydataset(Dataset):
    def __init__(self,root,train):
        super(Mydataset,self).__init__()
        self.root=root
    
        self.train=train
        if train:
            self.x=os.listdir(self.root)[0:30]
        else:
            self.x=os.listdir(self.root)[30:]
        
    def __len__(self):
        if self.train:
            return 30*2426
        return 19*2426
    def __getitem__(self,idx):
        index=idx//2426
        file=self.x[index]
        idx=idx%2426
        df=pd.read_csv(self.root+"/"+file)
        x_value=df.iloc[idx:idx+30][["Close","Open","Prev Close"]]
        x_value=np.array(x_value).reshape(-1)
        y_value=df.iloc[idx+30]["Open"]
        y_value=np.array(y_value)
        x_value=torch.tensor(x_value)
        y_value=torch.tensor(y_value)
        return x_value,y_value
        
        
        
        
    

In [ ]:
#loading data
train_dataset=Mydataset(root="mydata",train=True)
train_loader=DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True)
test_dataset=Mydataset(root="mydata",train=False,)
test_loader=DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=False)

In [ ]:
#defining network
class network(nn.Module):
    def __init__(self,input_dim,hidden_dim,output_dim):
        super(network,self).__init__()
        self.fc1=nn.Linear(input_dim,102)
        self.fc2=nn.Linear(102,hidden_dim)
        self.fc3=nn.Linear(hidden_dim,28)
        self.fc4=nn.Linear(28,1)
    def forward(self,x):
       
        out=self.fc1(x)
        out=self.fc2(out)
        out=self.fc3(out)
        out=self.fc4(out)
        return out
    
model=network(input_dim,hidden_dim,output_dim).to(device)    
    
        

In [ ]:
#define loss func and optim
criterion=nn.L1Loss()
optim=torch.optim.Adam(model.parameters(),lr=learning_rate)

In [ ]:
#training
loss_g=[]
for epoch in range(num_epochs):
    for i,(x_values,y_values) in enumerate(train_loader):
        x_values=x_values.reshape(-1,input_dim).to(device)
        y_values=y_values.float().to(device)
        pred=model(x_values.float())
        y_values=y_values.reshape(pred.shape)
        loss=criterion(pred,y_values)
        optim.zero_grad()
        loss.backward()
        optim.step()
        if i%100==0:
            print("epoch :[{}/{}] loss:[{}]".format(epoch+1,num_epochs,loss.item()))
            loss_g.append(loss.item())
            
        
        

In [ ]:
plt.plot(loss_g)

plt.show()

In [ ]:
y_gvalues=[]
pred_values=[]
for i in range (4000,4300):
    pred=model(test_dataset[i][0].float().to(device))
    pred_values.append(pred)
    y_gvalues.append(test_dataset[i][1])
plt.plot(y_gvalues,color="green",label="price actual")
plt.plot(pred_values,color="blue",label="price pred")

plt.legend()
plt.show()
